In [2]:
# Importation
import numpy as np
import pandas as pd
import plotly.express as px
from  rouge_score  import  rouge_scorer
from sklearn.metrics import cohen_kappa_score

# **I - Data management**

In [4]:
# Chargement des données
df = pd.read_csv("../data/01_251021_Notations200.csv")

In [34]:
df = pd.read_csv("C:/Users/Yanni/OneDrive/Desktop/projet d4g/scores_pour_annotation.csv") # Pour moi pcq je galere

## Présentation des données 


Le jeu de données qu'on traite dans le notebook est issue du jeu de données open source  [La fiscalité et les dépenses publiques](https://www.data.gouv.fr/fr/datasets/r/bc085888-e6bd-445d-b3f4-632190c29e3f). Celui-ci est notament constitué des réponses textuelles des contribuables à des questions portant sur la fiscalité en France. Dans cette étude, on se penche principalement sur la question suivante  : *Que faudrait-il faire pour rendre la fiscalité plus juste et plus efficace ?*.

Pour extraire idées majeures des réponses textuelles, on utilise le modèle de langue ollama : *hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4*. Avec les hyperparamètres **top_p = 0.95**
et **temperature = 0**.

On donne à ce LM le prompt suivant :


[prompt]
system = """
But: extraire les idées principales DISTINCTES d’un texte pour analyse.

Règles:
1. N’utiliser QUE le contenu entre <<< TEXT >>>.
2. Extraire la liste des idées DISTINCTES et PRINCIPALES.
   - Chaque idée = une phrase claire, autonome, reformulée si nécessaire.
3. Pour CHAQUE idée, annoter:
   - type: "statement" (constat) OU "proposition" (suggestion/recommandation/objectif).
   - syntax: "negative" si la phrase contient une négation explicite (ex.: "ne", "n'", "ne pas", "ne plus", "non"), sinon "positive".
   - semantic: "positive", "negative" ou "neutral" (valence sémantique).
4. Sortie STRICTEMENT en CSV avec entête EXACTE:
   CSV:description,type,syntax,semantic
   - Délimiteur: virgule.
   - Chaque description entre guillemets doubles.
   - Échapper tout guillemet interne par duplication (ex.: ""chat"").
   - NE RIEN AJOUTER d’autre (pas de texte avant/après, pas de code fences).
   - Pas de lignes vides.

Exemple:
CSV:description,type,syntax,semantic
"Les chats retombent sur leurs pattes",statement,positive,neutral
"Les chats n'ont pas neuf vies",statement,negative,negative
"Il faut mieux prendre soin des chats pour prolonger leur vie",proposition,positive,positive
"""
user = """<<< {input} >>>"""

Ainsi, on construit le jeu de données **df**. Ces vqriqbles sont les suivantes :

- **authorId** : Identifiant de l’auteur
- **contrib_index** : Numéro de la contribution
- **contribution** : Texte original de l'auteur
- **C** : Score calculé par la métrique *QualIT*
- **n_ideas** : Nombre d’idées extraites
- **len_contrib** : Longueur du texte original
- **Matthias, Yannis, Garance** : Note allant de 0 à 10 évaluant selon la personne la qualité de l'extraction
- **Hallucinations** : Présence d'hallucination dans l'extraction
- **Idées_non_ind** : Idées dénuée de sens dans l'extraction

In [5]:
# Ajout de nouvelles colonnes
### Calcul du score humain moyen
df["score_humain"] = df[["Garance", "Matthias", "Yannis"]].mean(axis=1)

### Nombre de charactères par contribution et par extractions
df["nb_char_contrib"] = df["contribution"].apply(len)
df["nb_char_extraction"] = df["ideas_text"].apply(len)

### Hallucinations et Idées_non_ind en catégories
df['hal_cat'] = df['Hallucinations'].astype("category")
df["idees_cat"] = df["Idées_non_ind"].astype("category")

### Catégories de longueur
length_bins = [0, 250, 500, 2000, float("inf")]
length_labels = ["Très courte (0-250)", "Courte (251-500)", "Moyenne (501-2000)", "Longue (2000+)"]
df["contrib_cat"] = pd.cut(df["nb_char_contrib"], bins=length_bins, labels=length_labels)

# **II - Validité de la notation et de la métrique**

In [4]:

# Calcul du kappa de Cohen entre chaque paire d'évaluateurs
kappa_GM = cohen_kappa_score(df["Garance"], df["Matthias"])
kappa_GY = cohen_kappa_score(df["Garance"], df["Yannis"])
kappa_MY = cohen_kappa_score(df["Matthias"], df["Yannis"])
print(f"Kappa Cohen entre Garance et Matthias : {kappa_GM:.2f}")
print(f"Kappa Cohen entre Garance et Yannis : {kappa_GY:.2f}")
print(f"Kappa Cohen entre Matthias et Yannis : {kappa_MY:.2f}")

Kappa Cohen entre Garance et Matthias : 0.34
Kappa Cohen entre Garance et Yannis : 0.42
Kappa Cohen entre Matthias et Yannis : 0.44


On recalcule le Kappa de Cohen mais cette fois-ci pondéré avec l'argument **weights="linear"** pour chaque paire d'évaluateur :

In [5]:

# Kappa pondéré linear entre chaque paire d'évaluateurs
kappa_GM = cohen_kappa_score(df["Garance"], df["Matthias"], weights='linear')
kappa_GY = cohen_kappa_score(df["Garance"], df["Yannis"], weights='linear')
kappa_MY = cohen_kappa_score(df["Matthias"], df["Yannis"], weights='linear')

print(f"Kappa pondéré (linear) entre Garance et Matthias : {kappa_GM:.2f}")
print(f"Kappa pondéré (linear) entre Garance et Yannis : {kappa_GY:.2f}")
print(f"Kappa pondéré (linear) entre Matthias et Yannis : {kappa_MY:.2f}")

# Bien plus significatif !!

Kappa pondéré (linear) entre Garance et Matthias : 0.77
Kappa pondéré (linear) entre Garance et Yannis : 0.79
Kappa pondéré (linear) entre Matthias et Yannis : 0.81


Il semble déjà y avoir un désaccord entre les évaluateurs humains. Les $\kappa$ de Cohen entre chaque paire d'évaluateurs se situent autour de 0.4, ce qui indique un accord faible/modéré selon l'échelle de Landis et Koch (*The measurement of observer agreement for categorical data*, 1977). \
NB : Un désaccord total ou un accord dû uniquement au hasard donnerait un $\kappa \leq 0$.

L'intérêt d'une triple notation est donc confirmé ; cela permet de lisser les divergence entre évaluateurs.

on peut maintenant regarder la corrélation entre le score humain moyen et le score QualIT. Les 2 variables sont continues, on utilise donc la corrélation de Pearson.

In [6]:
# Corrélation de Pearson entre score humain et score QualIT
correlation = df["score_humain"].corr(df["C"])
print(f"Corrélation entre le score humain moyen et le score QualIT : {correlation:.2f}")

Corrélation entre le score humain moyen et le score QualIT : 0.71


La corrélation entre le score humain moyen et la métrique QualIT est assez élevée ***ET*** positive, ce qui suggère qu'elle n'est pas trop mauvaise pour évaluer la qualité de l'extraction. Toutefois, ce résultat doit être interprété avec prudence pour 2 raisons :
- Le désaccord entre évaluateurs humains indique que la notation n'est pas totalement fiable.
- La métrique ne tient pas compte de la pertinence des idées extraites, seulement de leurs similarités avec la contribution originale.

# **III - Visualisations**

### **A - Caractéristiques des contributions**

In [6]:
# Barplot du nombre de charactères par contribution sur les 200 premières contributions
fig = px.histogram(
    df, x="nb_char_contrib", nbins=max(df["nb_char_contrib"])//100,
    color_discrete_sequence=["#2a2781"]
)
fig.update_layout(
    title={"text": "<b>Distribution du nombre de charactères par contribution</b>"}, 
    xaxis_title="Nombre de charactères", 
    yaxis_title="Nombre de contributions", 
    width=800, height=450
)
fig.update_traces(
    hovertemplate =
        "Nombre de charactères = %{x}<br>" \
        "Nombre de contributions = %{y}<extra></extra>"
)
fig.show()

Au vu de la distribution, il semble plus judicieux de visualiser sur les catégories de longueur (arbitraire).

### **B - Caractéristiques des extractions**

In [9]:
fig = px.histogram(
    df, x="contrib_cat", y="Hallucinations",
    color_discrete_sequence=["#2a2781"], histfunc="sum", category_orders=dict(contrib_cat=length_labels)
)
fig.update_layout(
    title={"text": "<b>Distribution du nombre d'hallucinations en fonction du nombre de <br>charactères par contribution</b>"}, 
    xaxis_title="Nombre de charactères", 
    yaxis_title="Nombre d'hallucinations", 
    width=800, height=450
)
fig.update_traces(
    hovertemplate =
        "Nombre d'hallucinations = %{y}<extra></extra>"
)
fig.show()

# Pour aller plus loin : afficher le nombre de contributions dans l'info-bulle.

In [10]:
fig = px.histogram(
    df, x="contrib_cat", y="Idées_non_ind",
    color_discrete_sequence=["#2a2781"], histfunc="sum", category_orders=dict(contrib_cat=length_labels)
)
fig.update_layout(
    title={"text": "<b>Distribution du nombre d'idées non indépendantes en fonction du <br>nombre de charactères par contribution</b>"}, 
    xaxis_title="Nombre de charactères", 
    yaxis_title="Nombre d'idées non indépendantes", 
    width=800, height=450
)
fig.update_traces(
    hovertemplate =
        "Nombre d'idées non indépendantes = %{y}<extra></extra>"
)
fig.show()

# Pour aller plus loin : afficher le nombre de contributions dans l'info-bulle.

### **C - Caractéristiques des évaluations**

In [12]:
fig = px.scatter(
    df, x="C", y="score_humain", 
    color="hal_cat", color_discrete_sequence=["#ff6361", "#003f5c"]
)
# Ajout de la ligne x = y pour référence
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>Comparaison du score humain avec la métrique QualIT</b>"},
    xaxis_title="Métrique QualIT",
    yaxis_title="Score humain",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
fig.show()

In [13]:
fig = px.scatter(
    df, x="C", y="score_humain", 
    color="idees_cat", color_discrete_sequence=["#ff6361", "#003f5c"]
)
# Ajout de la ligne x = y pour référence
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>Comparaison du score humain avec la métrique QualIT</b>"},
    xaxis_title="Métrique QualIT",
    yaxis_title="Score humain",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
fig.show()

In [14]:
df["log_score_humain"] = np.log10(df["score_humain"]+1)

fig = px.scatter(
    df, x="C", y="log_score_humain", 
    color="idees_cat", color_discrete_sequence=["#ff6361", "#003f5c"]
)
# Ajout de la ligne x = y pour référence
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=1, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>Comparaison du score humain avec la métrique QualIT</b>"},
    xaxis_title="Métrique QualIT",
    yaxis_title="Score humain",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
fig.show()

Les extractions échouées ne sont pas sur les graphiques précédents car elles n'ont pas de valeur dans "Hallucinations" ou "Idées_non_ind".

In [15]:
fig = px.scatter(
    df, x="C", y="score_humain", 
    color="contrib_cat", color_discrete_sequence=["#ffd380", "#ff6361", "#8a508f", "#00202e"]
)
# Ajout de la ligne x = y pour référence
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>Comparaison du score humain avec la métrique QualIT</b>"},
    xaxis_title="Métrique QualIT",
    yaxis_title="Score humain",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
fig.show()

In [16]:
fig = px.histogram(
    df, x="nb_char_contrib", y="C", nbins=max(df["nb_char_contrib"])//100,
    color_discrete_sequence=["#2a2781"], histfunc="avg"
)
fig.update_layout(
    title={"text": "<b>Métrique QualIT en fonction du nombre de charactères par contribution<b>"}, 
    xaxis_title="Nombre de charactères", 
    yaxis_title="Moyenne QualIT", 
    width=800, height=450
)
fig.update_traces(
    hovertemplate =
        "Nombre de charactères = %{x}<br>" \
        "Moyenne QualIT = %{y}<extra></extra>"
)
fig.show()

In [17]:
fig = px.histogram(
    df, x="nb_char_extraction", y="C", nbins=max(df["nb_char_extraction"])//50,
    color_discrete_sequence=["#2a2781"], histfunc="avg"
)
fig.update_layout(
    title={"text": "<b>Métrique QualIT en fonction du nombre de charactères par extraction<b>"}, 
    xaxis_title="Nombre de charactères", 
    yaxis_title="Moyenne QualIT", 
    width=800, height=450
)
fig.update_traces(
    hovertemplate =
        "Nombre de charactères = %{x}<br>" \
        "Moyenne QualIT = %{y}<extra></extra>"
)
fig.show()

## Métrique ROUGE

les métrique ROUGE (Recall-Oriented Understudy for Gisting Evaluation ) est un ensemble de métriques qui sont généralement utilisées pour évaluier la qualité des résumés proposées par un LLM. Ces différentes métriques repose dans notre cas sur les similarités entre les mots de l'extraction du LLM et l'idée de base. On peut définir 3 catégories de métriques ROUGE :
- N-grammes : compare les groupes de N mots consécutifs entre l'idée originel et l'extraction
- L ( Longest Common Subsequence) : mesure la similarité en séquence plutot qu'en N-gramm
- S/SU : Statistique de co-occurence reposant sur les bigrammes de saut (paire de mots quelconque dans leur ordre de phrase): bigramme sauté et les unigrammes

Celles-ci sont comprises entre 0 et 1 : plus la métrique est proche de 1 et plus l'extraction est de bonne qualité. Inversement, plus la métrique est proche de 0 et plus elle est de mauvaise qualité.

Dans notre cas, on cherche à évaluer la qualité des idées extraites plus qu'une similarité mot à mot. La métrique la plmus adapté semble donc etre la métrique LCS. 
Pour implémenter cette métrique avec Python, on emploiera le package [rouge-score](https://pypi.org/project/rouge-score/). L'exemple de base sur ce site calcule la métrique rouge &-gramm et L, ce que l'on va calculer puis implémenter dans le jeud e donnée df 

In [7]:
# On crée les vars pour la métrique ROUGE
df["1gramm"],df["L"] = 0,0

In [8]:

# Les 2 métriques à évaluer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

for i in range(np.shape(df)[0]):
    # try pour que ca passe avec les parse error
    try:
        ref = str(df.loc[i, 'ideas_text'])
        pred = str(df.loc[i, 'contribution'])
        
        scores = scorer.score(ref, pred)
        
        df.loc[i, '1gramm'] = scores['rouge1'].fmeasure
        df.loc[i, 'L'] = scores['rougeL'].fmeasure
    
    # 0 si ca ne passe pas
    except Exception as e:
        print(f"Erreur à la ligne {i} : {e}")
        df.loc[i, '1gramm'] = 0
        df.loc[i, 'L'] = 0




/var/folders/9x/5wd13t7x7377s56tp32vtnnr0000gn/T/ipykernel_5333/2065271912.py:12: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0014492753623188406' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/var/folders/9x/5wd13t7x7377s56tp32vtnnr0000gn/T/ipykernel_5333/2065271912.py:13: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0014492753623188406' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [9]:
# test
sum(df["1gramm"].isnull())
sum(df["L"].isnull())

0

In [10]:
# Corrélation de Pearson entre score humain et score ROUGE &gramm/L
correlation_1gramm = df["score_humain"].corr(df["1gramm"])
print(f"Corrélation entre le score humain moyen et le score ROUGE 1gramm : {correlation_1gramm:.2f}")

correlation_L = df["score_humain"].corr(df["L"])
print(f"Corrélation entre le score humain moyen et le score ROUGE L : {correlation_L:.2f}")

Corrélation entre le score humain moyen et le score ROUGE 1gramm : 0.79
Corrélation entre le score humain moyen et le score ROUGE L : 0.78


Les corrélation entre l'évaluation humaine et celles des métriques ROUGE est plus grande que celle de QualIT (0.71)

## Plot des métriques rouges ( copie de ce qu'à fait Matthias concrétement)

In [11]:
fig = px.scatter(
    df, x="1gramm", y="score_humain", 
    color="hal_cat", color_discrete_sequence=["#ff6361", "#003f5c"]
)
# Ajout de la ligne x = y pour référence
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>Comparaison du score rouge avec la métrique ROUGE 1-gramm</b>"},
    xaxis_title="Métrique ROUGE 1-gramm",
    yaxis_title="Score humain",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "ROUGE 1-gramm = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
fig.show()

In [12]:
fig = px.scatter(
    df, x="L", y="score_humain", 
    color="hal_cat", color_discrete_sequence=["#ff6361", "#003f5c"]
)
# Ajout de la ligne x = y pour référence
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>Comparaison du score rouge avec la métrique ROUGE L</b>"},
    xaxis_title="Métrique ROUGE L",
    yaxis_title="Score humain",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "ROUGE L = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
fig.show()

In [13]:
fig = px.scatter(
    df, x="1gramm", y="score_humain", 
    color="contrib_cat", color_discrete_sequence=["#ffd380", "#ff6361", "#8a508f", "#00202e"]
)
# Ajout de la ligne x = y pour référence
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>Comparaison du score humain avec la métrique ROUGE 1gramm</b>"},
    xaxis_title="Métrique QualIT",
    yaxis_title="Score humain",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "Métrique ROUGE 1gramm = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
fig.show()

In [41]:
fig = px.scatter(
    df, x="L", y="score_humain", 
    color="contrib_cat", color_discrete_sequence=["#ffd380", "#ff6361", "#8a508f", "#00202e"]
)
# Ajout de la ligne x = y pour référence
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>Comparaison du score humain avec la métrique ROUGE 1gramm</b>"},
    xaxis_title="Métrique ROUGE L",
    yaxis_title="Score humain",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "Métrique ROUGE L = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
fig.show()